In [1]:
import h5py
import torch
import numpy as np
import os

ori_path = r'./datasets/trainnew'
out_path = r'./datasets/trainnew_h5'
all_subject = os.listdir(ori_path)

FileNotFoundError: [Errno 2] No such file or directory: './datasets/trainnew'

In [2]:
import torch.nn.functional as F
from utils.parse_image_file import parse_image
from utils.blur_kernel_ops import calc_extended_patch_size, parse_kernel
from utils.pad import target_pad

/home/songzhiyun/software/anaconda3/envs/segsr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def preprocess(subject):
    image, slice_separation, _,  blur_fwhm, _, _, _, _ = parse_image(
        os.path.join(ori_path, subject), 3.0, 0.75
    )
    image = image.squeeze() # shape (x, y, z, 2)
    if len(image.shape) == 3:
        image = image[..., np.newaxis]
    
    # upsample the image using b-spline interpolation
    img_data = image[...,0]
    label_data = image[...,1]
    upsampled_img_data = scipy.ndimage.zoom(img_data, (1,1,4), order=3)  # order=3 for cubic B-spline
    upsampled_label_data = scipy.ndimage.zoom(label_data, (1,1,4), order=0)  # order=0 for nearest interpolation
    image = np.stack([upsampled_img_data, upsampled_label_data], axis=-1)
    
    blur_kernel = parse_kernel(None, 'rf-pulse-slr', blur_fwhm)
    img_hr = image[...,:1]
    label_hr = image[...,1:].astype('uint8')
    image_x = torch.from_numpy(image.transpose(2, 3, 0, 1)) # z, channel, x, y
    image_x_rgb = image_x[:, 0:1, ...]
    image_x_rgb = F.conv2d(image_x_rgb, blur_kernel, padding="same").numpy()

    image_y = torch.from_numpy(image.transpose(2, 3, 1, 0)) # z, channel, y, x
    image_y_rgb = image_y[:, 0:1, ...]
    image_y_rgb = F.conv2d(image_y_rgb, blur_kernel, padding="same").numpy()
    return img_hr, label_hr, image_x_rgb, image_y_rgb

In [4]:
for subject in all_subject:
    img_hr, label_hr, image_x_rgb, image_y_rgb = preprocess(subject)
    with h5py.File(os.path.join(out_path, subject + '.h5'), 'w') as f:
        f.create_dataset('img_hr', data=img_hr)
        f.create_dataset('label_hr', data=label_hr)
        f.create_dataset('image_x_rgb', data=image_x_rgb)
        f.create_dataset('image_y_rgb', data=image_y_rgb)

In [4]:
def zeroonenorm(data):
    data =  (data - np.min(data)) / (np.max(data) - np.min(data))
    data = (data * 255.0)
    return data

In [5]:
def rescale_intensity(volume, percentils=[0.5, 99.5], bins_num=256, norm=False):
    volume[volume < 0] = 0
    obj_volume = volume[np.where(volume > 0)]
    min_value = np.percentile(obj_volume, percentils[0])
    max_value = np.percentile(obj_volume, percentils[1])

    if bins_num == 0:
        obj_volume = (obj_volume - min_value) / (max_value - min_value).astype(np.float32)
    else:
        obj_volume = np.round((obj_volume - min_value) / (max_value - min_value) * (bins_num - 1))
        obj_volume[np.where(obj_volume < 1)] = 1
        obj_volume[np.where(obj_volume > (bins_num - 1))] = bins_num - 1

    volume[np.where(volume > 0)] = obj_volume
    volume = volume.astype(obj_volume.dtype)
    if norm:
        volume = volume.astype(float) / (bins_num - 1)

    return volume

In [6]:
import nibabel as nib
from scipy.ndimage import gaussian_filter
from scipy.interpolate import RegularGridInterpolator

def get_dims(shape, max_channels=10):
    """Get the number of dimensions and channels from the shape of an array.
    The number of dimensions is assumed to be the length of the shape, as long as the shape of the last dimension is
    inferior or equal to max_channels (default 3).
    :param shape: shape of an array. Can be a sequence or a 1d numpy array.
    :param max_channels: maximum possible number of channels.
    :return: the number of dimensions and channels associated with the provided shape.
    example 1: get_dims([150, 150, 150], max_channels=10) = (3, 1)
    example 2: get_dims([150, 150, 150, 3], max_channels=10) = (3, 3)
    example 3: get_dims([150, 150, 150, 15], max_channels=10) = (4, 1), because 5>3"""
    if shape[-1] <= max_channels:
        n_dims = len(shape) - 1
        n_channels = shape[-1]
    else:
        n_dims = len(shape)
        n_channels = 1
    return n_dims, n_channels

def get_ras_axes(aff, n_dims=3):
    """This function finds the RAS axes corresponding to each dimension of a volume, based on its affine matrix.
    :param aff: affine matrix Can be a 2d numpy array of size n_dims*n_dims, n_dims+1*n_dims+1, or n_dims*n_dims+1.
    :param n_dims: number of dimensions (excluding channels) of the volume corresponding to the provided affine matrix.
    :return: two numpy 1d arrays of length n_dims, one with the axes corresponding to RAS orientations,
    and one with their corresponding direction.
    """
    aff_inverted = np.linalg.inv(aff)
    img_ras_axes = np.argmax(np.absolute(aff_inverted[0:n_dims, 0:n_dims]), axis=0)
    for i in range(n_dims):
        if i not in img_ras_axes:
            unique, counts = np.unique(img_ras_axes, return_counts=True)
            incorrect_value = unique[np.argmax(counts)]
            img_ras_axes[np.where(img_ras_axes == incorrect_value)[0][-1]] = i

    return img_ras_axes

def align_volume_to_ref(volume, aff, aff_ref=None, return_aff=False, n_dims=None, return_copy=True):
    """This function aligns a volume to a reference orientation (axis and direction) specified by an affine matrix.
    :param volume: a numpy array
    :param aff: affine matrix of the floating volume
    :param aff_ref: (optional) affine matrix of the target orientation. Default is identity matrix.
    :param return_aff: (optional) whether to return the affine matrix of the aligned volume
    :param n_dims: (optional) number of dimensions (excluding channels) of the volume. If not provided, n_dims will be
    inferred from the input volume.
    :param return_copy: (optional) whether to return the original volume or a copy. Default is copy.
    :return: aligned volume, with corresponding affine matrix if return_aff is True.
    """

    # work on copy
    new_volume = volume.copy() if return_copy else volume
    aff_flo = aff.copy()

    # default value for aff_ref
    if aff_ref is None:
        aff_ref = np.eye(4)

    # extract ras axes
    if n_dims is None:
        n_dims, _ = get_dims(new_volume.shape)
    ras_axes_ref = get_ras_axes(aff_ref, n_dims=n_dims)
    ras_axes_flo = get_ras_axes(aff_flo, n_dims=n_dims)

    # align axes
    aff_flo[:, ras_axes_ref] = aff_flo[:, ras_axes_flo]
    for i in range(n_dims):
        if ras_axes_flo[i] != ras_axes_ref[i]:
            new_volume = np.swapaxes(new_volume, ras_axes_flo[i], ras_axes_ref[i])
            swapped_axis_idx = np.where(ras_axes_flo == ras_axes_ref[i])
            ras_axes_flo[swapped_axis_idx], ras_axes_flo[i] = ras_axes_flo[i], ras_axes_flo[swapped_axis_idx]

    # align directions
    dot_products = np.sum(aff_flo[:3, :3] * aff_ref[:3, :3], axis=0)
    for i in range(n_dims):
        if dot_products[i] < 0:
            new_volume = np.flip(new_volume, axis=i)
            aff_flo[:, i] = - aff_flo[:, i]
            aff_flo[:3, 3] = aff_flo[:3, 3] - aff_flo[:3, i] * (new_volume.shape[i] - 1)

    if return_aff:
        return new_volume, aff_flo
    else:
        return new_volume

def load_volume(path_volume, im_only=True, squeeze=True, dtype=None, aff_ref=None):
    """
    Load volume file.
    :param path_volume: path of the volume to load. Can either be a nii, nii.gz, mgz, or npz format.
    If npz format, 1) the variable name is assumed to be 'vol_data',
    2) the volume is associated with an identity affine matrix and blank header.
    :param im_only: (optional) if False, the function also returns the affine matrix and header of the volume.
    :param squeeze: (optional) whether to squeeze the volume when loading.
    :param dtype: (optional) if not None, convert the loaded volume to this numpy dtype.
    :param aff_ref: (optional) If not None, the loaded volume is aligned to this affine matrix.
    The returned affine matrix is also given in this new space. Must be a numpy array of dimension 4x4.
    :return: the volume, with corresponding affine matrix and header if im_only is False.
    """
    assert path_volume.endswith(('.nii', '.nii.gz', '.mgz', '.npz')), 'Unknown data file: %s' % path_volume

    if path_volume.endswith(('.nii', '.nii.gz', '.mgz')):
        x = nib.load(path_volume)
        if squeeze:
            volume = np.squeeze(x.get_fdata())
        else:
            volume = x.get_fdata()
        aff = x.affine
        header = x.header
    else:  # npz
        volume = np.load(path_volume)['vol_data']
        if squeeze:
            volume = np.squeeze(volume)
        aff = np.eye(4)
        header = nib.Nifti1Header()
    if dtype is not None:
        if 'int' in dtype:
            volume = np.round(volume)
        volume = volume.astype(dtype=dtype)

    # align image to reference affine matrix
    if aff_ref is not None:
        n_dims, _ = get_dims(list(volume.shape), max_channels=10)
        volume, aff = align_volume_to_ref(volume, aff, aff_ref=aff_ref, return_aff=True, n_dims=n_dims)

    if im_only:
        return volume
    else:
        return volume, aff, header

def save_volume(volume, aff, header, path, dtype=None, n_dims=3):
    """
    Save a volume.
    :param volume: volume to save
    :param aff: affine matrix of the volume to save. If aff is None, the volume is saved with an identity affine matrix.
    aff can also be set to 'FS', in which case the volume is saved with the affine matrix of FreeSurfer outputs.
    :param header: header of the volume to save. If None, the volume is saved with a blank header.
    :param path: path where to save the volume.
    :param res: (optional) update the resolution in the header before saving the volume.
    :param dtype: (optional) numpy dtype for the saved volume.
    :param n_dims: (optional) number of dimensions, to avoid confusion in multi-channel case. Default is None, where
    n_dims is automatically inferred.
    """

    os.makedirs(os.path.dirname(path), exist_ok=True)
    if '.npz' in path:
        np.savez_compressed(path, vol_data=volume)
    else:
        if header is None:
            header = nib.Nifti1Header()
        if isinstance(aff, str):
            if aff == 'FS':
                aff = np.array([[-1, 0, 0, 0], [0, 0, 1, 0], [0, -1, 0, 0], [0, 0, 0, 1]])
        elif aff is None:
            aff = np.eye(4)
        nifty = nib.Nifti1Image(volume, aff, header)
        if dtype is not None:
            if 'int' in dtype:
                volume = np.round(volume)
            volume = volume.astype(dtype=dtype)
            nifty.set_data_dtype(dtype)
        nib.save(nifty, path)

def resample_volume(volume, aff, new_vox_size, interpolation='linear', blur=True):
    """This function resizes the voxels of a volume to a new provided size, while adjusting the header to keep the RAS
    :param volume: a numpy array
    :param aff: affine matrix of the volume
    :param new_vox_size: new voxel size (3 - element numpy vector) in mm
    :param interpolation: (optional) type of interpolation. Can be 'linear' or 'nearest. Default is 'linear'.
    :return: new volume and affine matrix
    """

    pixdim = np.sqrt(np.sum(aff * aff, axis=0))[:-1]
    new_vox_size = np.array(new_vox_size)
    factor = pixdim / new_vox_size
    sigmas = 0.25 / factor
    sigmas[factor > 1] = 0  # don't blur if upsampling
    if not blur:
        sigmas = (0,0,0)

    volume_filt = gaussian_filter(volume, sigmas)

    # volume2 = zoom(volume_filt, factor, order=1, mode='reflect', prefilter=False)
    x = np.arange(0, volume_filt.shape[0])
    y = np.arange(0, volume_filt.shape[1])
    z = np.arange(0, volume_filt.shape[2])

    my_interpolating_function = RegularGridInterpolator((x, y, z), volume_filt, method=interpolation)

    start = - (factor - 1) / (2 * factor)
    step = 1.0 / factor
    stop = start + step * np.ceil(volume_filt.shape * factor)

    xi = np.arange(start=start[0], stop=stop[0], step=step[0])
    yi = np.arange(start=start[1], stop=stop[1], step=step[1])
    zi = np.arange(start=start[2], stop=stop[2], step=step[2])
    xi[xi < 0] = 0
    yi[yi < 0] = 0
    zi[zi < 0] = 0
    xi[xi > (volume_filt.shape[0] - 1)] = volume_filt.shape[0] - 1
    yi[yi > (volume_filt.shape[1] - 1)] = volume_filt.shape[1] - 1
    zi[zi > (volume_filt.shape[2] - 1)] = volume_filt.shape[2] - 1

    xig, yig, zig = np.meshgrid(xi, yi, zi, indexing='ij', sparse=True)
    volume2 = my_interpolating_function((xig, yig, zig))

    aff2 = aff.copy()
    for c in range(3):
        aff2[:-1, c] = aff2[:-1, c] / factor[c]
    aff2[:-1, -1] = aff2[:-1, -1] - np.matmul(aff2[:-1, :-1], 0.5 * (factor - 1))

    return volume2, aff2

In [8]:
import h5py
import torch
import numpy as np
import os

ori_path = r"/mnt/INSPUR_storage/songzhiyun/project/BoneTumorSeg/data/Brain_datasets/bspline_sr"
tmp_path = r'/home/songzhiyun/project/BoneTumorSeg/data/Liver_datasets/trainnew_sr_resample_temp'
out_path = r'/home/songzhiyun/project/BoneTumorSeg/data/Brain_datasets/bspline_sr_h5'
os.makedirs(tmp_path, exist_ok=True)
all_subject = os.listdir(ori_path)
all_subject = [x for x in all_subject if x.endswith('_img.nii.gz')]
for subject in all_subject:
    # ori_t1_data, aff_t1, hdr = load_volume(os.path.join(ori_path, subject), im_only=False, dtype='float')
    # pixdim = np.sqrt(np.sum(aff_t1 * aff_t1, axis=0))[:-1]
    # t1_data_lr, aff_t1_lr = resample_volume(ori_t1_data, aff_t1, [1.04167,1.04167,1.5])
    # save_volume(t1_data_lr, aff_t1_lr, hdr, os.path.join(tmp_path, subject))

    # ori_t2_data, aff_t2, hdr = load_volume(os.path.join(ori_path, subject.replace('_img', '_seg')), im_only=False, dtype='uint8')
    # t2_data_lr, aff_t2_lr = resample_volume(ori_t2_data, aff_t2, [1.04167,1.04167,1.5], interpolation='nearest')
    # save_volume(t2_data_lr, aff_t2_lr, hdr, os.path.join(tmp_path, subject.replace('_img', '_seg')))

    # ori_t3_data, aff_t3, hdr = load_volume(os.path.join(ori_path, subject.replace('_img', '_uncertainty')), im_only=False, dtype='float')
    # t3_data_lr, aff_t3_lr = resample_volume(ori_t3_data, aff_t3, [1.04167,1.04167,1.5])
    # save_volume(t3_data_lr, aff_t3_lr, hdr, os.path.join(tmp_path, subject.replace('_img', '_uncertainty')))
    

    image, _, _,  _, _, _, _, _ = parse_image(
        os.path.join(ori_path, subject), 4.0, 1.0
    )
    image = zeroonenorm(image)
    seg, _, _,  _, _, _, _, _ = parse_image(
        os.path.join(ori_path, subject.replace('_img', '_seg')), 4.0, 1.0
    )
    seg = seg.astype('uint8')
    if os.path.exists(os.path.join(ori_path, subject.replace('_img', '_uncertainty'))):
        uncertainty, _, _,  _, _, _, _, _ = parse_image(
            os.path.join(ori_path, subject.replace('_img', '_uncertainty')), 4.0, 1.0
        )
        uncertainty = (zeroonenorm(uncertainty) * 255.0).astype('uint8')
    else:
        uncertainty = np.zeros_like(seg)
    
    with h5py.File(os.path.join(out_path, subject.replace('_img.nii.gz', '_0000.h5')), 'w') as f:
        f.create_dataset('img', data=image)
        f.create_dataset('seg', data=seg)
        f.create_dataset('uncertainty', data=uncertainty)


In [14]:
import h5py
import torch
import numpy as np
import os

ori_path = r'/home/songzhiyun/project/BoneTumorSeg/code/nnUNet/DATASET/nnUNet_raw/Dataset510_BoneTumor/imagesTr'
tmp_path = r'/home/songzhiyun/project/BoneTumorSeg/code/nnUNet/DATASET/nnUNet_raw/Dataset510_BoneTumor/imagesTr'
os.makedirs(tmp_path, exist_ok=True)
all_subject = os.listdir(ori_path)
all_subject = [x for x in all_subject if x.endswith('.nii.gz')]
for subject in all_subject:
    ori_t1_data, aff_t1, hdr = load_volume(os.path.join(ori_path, subject), im_only=False, dtype='float')
    pixdim = np.sqrt(np.sum(aff_t1 * aff_t1, axis=0))[:-1]
    t1_data_lr, aff_t1_lr = resample_volume(ori_t1_data, aff_t1, [1.04167,1.04167,1.5])
    save_volume(t1_data_lr, aff_t1_lr, hdr, os.path.join(tmp_path, subject))

    ori_t2_data, aff_t2, hdr = load_volume(os.path.join(ori_path.replace('imagesTr', 'labelsTr'), subject.replace('_0000', '')), im_only=False, dtype='uint8')
    t2_data_lr, aff_t2_lr = resample_volume(ori_t2_data, aff_t2, [1.04167,1.04167,1.5], interpolation='nearest')
    save_volume(t2_data_lr, aff_t2_lr, hdr, os.path.join(ori_path.replace('imagesTr', 'labelsTr'), subject.replace('_0000', '')))

    

In [17]:
subject

'116-dicom_0000_img.nii.gz'